# Grazioso Salvare Rescue Dashboard

In [1]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table, Input, Output
import pandas as pd
import plotly.express as px

from AnimalShelter_Local import AnimalShelter
from pymongo import MongoClient
import os
os.environ['FLASK_ENV'] = 'development'

shelter = AnimalShelter()
test_data = shelter.read({})
print(f"Connected! Found {len(test_data)} records.")

# MongoDB connection
client = MongoClient('localhost', 27017)
db = client['AAC']
collection = db['animals']


df = pd.read_csv('aac_shelter_outcomes.csv')
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()
shelter = AnimalShelter()

data = shelter.read()
df = pd.DataFrame(data)




Connected! Found 10000 records.


In [2]:

empty_df = pd.DataFrame()

app = JupyterDash(__name__)

app.title = "Grazioso Salvare Dashboard"

app.layout = html.Div([
    html.A(
        html.Img(src='/assets/Grazioso Salvare Logo.png', style={'width': '200px'}),
        href="https://www.snhu.edu",
        target="_blank"
    ),

    html.H1("Grazioso Salvare Rescue Dog Dashboard", style={'textAlign': 'center'}),

    html.Div([
        html.Label("Select Rescue Type:"),
        dcc.Dropdown(
            id='rescue-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset'
        ),

            dcc.Dropdown(
            id='sex-dropdown',
            options=[{'label': s, 'value': s} for s in ['Male', 'Female']],
            value='Male'
        ),
            dcc.Dropdown(
            id='age-dropdown',
            options=[{'label': a, 'value': a} for a in ['Puppy', 'Young', 'Adult', 'Senior']],
            value='Adult'
        ),

    ], style={'width': '50%', 'margin': 'auto'}),

    html.Br(),

    dash_table.DataTable(
        id='datatable',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=empty_df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        style_header={'fontWeight': 'bold'}
    ),

     

    html.Br(),

    dcc.Graph(id='map-chart'),
    html.Br(),
    dcc.Graph(id='second-chart'),

    html.Footer("Created by Lara Walker", style={'textAlign': 'center', 'marginTop': '40px'})
])


/Users/student11/Desktop/ProjectTwoFolder/.venv/lib/python3.10/site-packages/dash/dash.py:587: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [3]:
@app.callback(
    [Output('datatable', 'data'),
     Output('map-chart', 'figure'),
     Output('second-chart', 'figure')],
    [Input('rescue-type', 'value'),
     Input('sex-dropdown', 'value'),
     Input('age-dropdown', 'value')]
)
def update_dashboard(rescue_type, selected_sex, selected_age_group):
    # Age filter setup
    age_group_map = {
        "Puppy": (0, 26),
        "Young": (27, 52),
        "Adult": (53, 156),
        "Senior": (157, 520)
    }
    age_range = age_group_map.get(selected_age_group, (0, 520))

    # Sex filter setup
    sex_map = {
        "Female": ["Intact Female", "Spayed Female"],
        "Male": ["Intact Male", "Neutered Male"]
    }

    # Base query with age and sex
    query = {
        "age_upon_outcome_in_weeks": {
            "$gte": age_range[0],
            "$lte": age_range[1]
        },
        "sex_upon_outcome": {
            "$in": sex_map.get(selected_sex, [])
        }
    }

    # Add rescue-type-specific breed filters
    if rescue_type == "Water Rescue":
        query["breed"] = {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}
    elif rescue_type == "Mountain or Wilderness Rescue":
        query["breed"] = {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}
    elif rescue_type == "Disaster or Individual Tracking":
        query["breed"] = {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}

    # Query MongoDB
    filtered_data = []
    for doc in shelter.read(query):
        doc['_id'] = str(doc['_id'])  # Convert ObjectId to string
        filtered_data.append(doc)

    filtered_df = pd.DataFrame(filtered_data)

    # Map chart
    if 'location_lat' in filtered_df.columns and 'location_long' in filtered_df.columns:
        map_fig = px.scatter_mapbox(
            filtered_df,
            lat='location_lat',
            lon='location_long',
            hover_name='breed',
            zoom=10
        )
        map_fig.update_layout(mapbox_style='open-street-map')
    else:
        map_fig = {}

    # Pie chart
    if 'breed' in filtered_df.columns:
        second_fig = px.pie(
            filtered_df,
            names='breed',
            title='Breed Distribution for Selected Rescue Type'
        )
    else:
        second_fig = {}

    return filtered_df.to_dict('records'), map_fig, second_fig






In [4]:


app.run(mode='external', debug=True)















---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.

---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.



/var/folders/nd/wy_g2njx03s0k_1m_336q4rh0000gn/T/ipykernel_23822/775169757.py:54: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.

---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
NotFound: 404 Not Found: The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.



/var/folders/nd/wy_g2njx03s0k_1m_336q4rh0000gn/T/ipykernel_23822/775169757.py:54: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/var/folders/nd/wy_g2njx03s0k_1m_336q4rh0000gn/T/ipykernel_23822/775169757.py:54: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/var/folders/nd/wy_g2njx03s0k_1m_336q4rh0000gn/T/ipykernel_23822/775169757.py:54: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

